In [13]:
import pandas as pd
from models_classifier import generate_model
import matplotlib.pyplot as plt
from xgboost import plot_importance
import shap
from functions import feature_importance
from preprocessing import split_df
import re

In [2]:
data_path = "./data/Preprocessed_Data.xlsx"
df = pd.read_excel(data_path)
df

,Age,Male,White,BMI,Tobacco Use,Diabetes,HbA1c Baseline,Hypertension,Hyperlipidemia,CKD,...,CK angle (deg)_rate_since_last_timepoint,CK MA (mm)_difference_since_last_timepoint,CK MA (mm)_rate_since_last_timepoint,CRT MA (mm)_difference_since_last_timepoint,CRT MA (mm)_rate_since_last_timepoint,CKH R (min)_difference_since_last_timepoint,CKH R (min)_rate_since_last_timepoint,CFF MA (mm)_difference_since_last_timepoint,CFF MA (mm)_rate_since_last_timepoint,Upcoming_event
0,70,1,0,29.728808,1,0,5,0,2,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
1,46,0,0,27.547356,3,1,8,1,2,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
2,67,1,1,31.391773,3,0,6,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
3,48,1,0,31.700002,1,0,6,1,1,0,...,-0.036173,0.681810,0.075757,-0.728237,-0.080915,0.222137,0.024682,-0.548554,-0.060950,False
4,57,1,1,27.309938,2,0,7,0,2,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,50,1,1,30.454834,3,0,7,1,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
146,59,0,1,29.891027,1,1,6,0,2,1,...,-0.513695,0.116933,0.038978,0.401491,0.133830,-0.267984,-0.089328,0.491565,0.163855,False
147,47,1,0,28.158641,3,0,5,0,0,1,...,-0.098728,-0.650629,-0.216876,-0.153692,-0.051231,0.167797,0.055932,1.633006,0.544335,False
148,55,1,1,27.659768,3,0,5,1,0,1,...,-0.051557,0.870876,0.290292,-0.151138,-0.050379,0.049269,0.016423,-0.721748,-0.240583,False


In [3]:
# Make models to find contribution of each parameter
best_model, best_params, accuracy, X_train = generate_model(df)


is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
is_sparse is deprecated and will be removed in a future version. Check `

In [4]:
model = best_model.named_steps['classifier']
scaler = best_model.named_steps['scaler']
model

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0.1, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=2, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [5]:
# Get feature importances from the XGBoost model in the pipeline
feature_importance_df = feature_importance(best_model, X_train)
feature_importance_df

,Feature,Importance,Percentage Contribution
36,MAPT,0.062027,6.202672
59,CKH R (min),0.056331,5.633118
64,Artery affected_Tibial,0.054973,5.497292
35,NAPT,0.047603,4.760325
11,History of MI,0.038460,3.845989
...,...,...,...
71,Reaction Time (R) in min_difference_since_last...,0.000000,0.000000
72,Reaction Time (R) in min_rate_since_last_timep...,0.000000,0.000000
73,Lysis at 30 min (LY30) in %_difference_since_l...,0.000000,0.000000
74,Lysis at 30 min (LY30) in %_rate_since_last_ti...,0.000000,0.000000


In [6]:
output_file = './data/Feature_importance.xlsx'
feature_importance_df.to_excel(output_file, index=False)


In [7]:
# Create a DataFrame with the features related to TEG values
teg_df , non_teg_df = split_df(feature_importance_df)
teg_df

,Feature,Importance,Percentage Contribution
52,AA % Aggregation,0.027300,2.729963
49,AA MA (mm),0.026512,2.651165
50,ADP % Aggregation,0.009106,0.910635
47,ActF MA (mm),0.019014,1.901362
60,CFF MA (mm),0.032130,3.212984
55,CK K (min),0.034672,3.467174
101,CK MA (mm)_difference_since_last_timepoint,0.000000,0.000000
54,CK R (min),0.018974,1.897378
100,CK angle (deg)_rate_since_last_timepoint,0.030225,3.022493
59,CKH R (min),0.056331,5.633118


In [8]:
non_teg_df

,Feature,Importance,Percentage Contribution
36,MAPT,0.062027,6.202672
64,Artery affected_Tibial,0.054973,5.497292
35,NAPT,0.047603,4.760325
11,History of MI,0.038460,3.845989
7,Hypertension,0.037809,3.780941
12,Functional Status,0.037318,3.731775
68,Anticoagulation_Warfarin,0.031751,3.175073
20,EGFR (mL/min/1.73m²),0.026261,2.626084
65,Extremity_Left,0.026251,2.625132
41,Visit Timepoint,0.023620,2.361958


In [16]:
# Select only the other features and the ones selected by user
selected_features = {
"Clot treatment drugs":"HKH MA (mm) (3.6591007709503174%)",
"Effects of fibrogen":"ActF MA (mm) (1.9013619422912598%)",
"Heparin effect":"HKH MA (mm) (3.6591007709503174%)",
"Unaltered blood ":"CK MA (mm)_difference_since_last_timepoint (0.0%)",
"Unaltered time to clot ":"CK R (min) (1.8973780870437622%)",
}


In [17]:
selected_features_list = []

# Process the values in the dictionary
for key, value in selected_features.items():
    # Remove patterns matching (float%)
    selected_features_list.append(re.sub(r'\(\d+\.\d+%\)', '', value).strip())
selected_features_list

['HKH MA (mm)',
 'ActF MA (mm)',
 'HKH MA (mm)',
 'CK MA (mm)_difference_since_last_timepoint',
 'CK R (min)']

In [18]:
# Extract the features mentioned in the "Feature" column of non_teg_df
non_teg_features = non_teg_df['Feature'].tolist()
selected_features_list =  selected_features_list + non_teg_features
selected_features_list

['HKH MA (mm)',
 'ActF MA (mm)',
 'HKH MA (mm)',
 'CK MA (mm)_difference_since_last_timepoint',
 'CK R (min)',
 'MAPT',
 'Artery affected_Tibial',
 'NAPT',
 'History of MI',
 'Hypertension',
 'Functional Status',
 'Anticoagulation_Warfarin',
 'EGFR (mL/min/1.73m²)',
 'Extremity_Left',
 'Visit Timepoint',
 'Bypass',
 'DAPT',
 'White',
 'DVA',
 'Aspirin Dose',
 'Hyperlipidemia',
 'ABI Left',
 'Does Subject Currently have cancer?',
 'ABI Right',
 'CFF FLEV (mg/dL)',
 'CKD',
 'Hx of stroke',
 'DOAC',
 'BMI',
 'Age',
 'Thrombectomy',
 'Cilostazol',
 'Tobacco Use',
 'Diabetes',
 'Statin',
 'Plavix Dose',
 'Endarterectomy',
 'Stent',
 'Angioplasty',
 'Rutherford Score',
 'Previous failed bypass',
 'Male',
 'Previous occluded stents',
 'Has subject had previous intervention of the index limb?',
 'Does the patient have a history of solid organ transplant?',
 'Hx of pulmonary embolism',
 'Hx of DVT',
 'Past hx of cancer',
 'Artery affected_Femoral',
 'Artery affected_Popliteal',
 'Clotting Disor

In [19]:
# Filter the columns of the original DataFrame based on the selected features
filtered_df = df[selected_features_list]
filtered_df

,HKH MA (mm),ActF MA (mm),HKH MA (mm),CK MA (mm)_difference_since_last_timepoint,CK R (min),MAPT,Artery affected_Tibial,NAPT,History of MI,Hypertension,...,Past hx of cancer,Artery affected_Femoral,Artery affected_Popliteal,Clotting Disorder,Coronary Artery Disease,Extremity_Right,Anticoagulation_Heparin,HbA1c Baseline,Intervention Classification_Endo,Intervention Classification_Open
0,47.428555,49.708250,47.428555,NaN,3.052992,0.922518,True,0.958164,1,0,...,1,False,False,0,1,False,True,5,False,True
1,49.603752,49.306924,49.603752,NaN,3.219205,0.900255,False,0.935640,1,1,...,0,True,False,0,1,True,False,8,False,True
2,47.120920,50.181848,47.120920,NaN,3.156927,0.913959,False,0.944335,0,0,...,0,False,True,0,0,True,False,6,False,True
3,49.195540,49.570241,49.195540,0.681810,3.110726,0.930112,True,0.918974,0,1,...,1,False,False,1,1,True,False,6,True,False
4,47.321239,51.502837,47.321239,NaN,3.313738,0.912491,True,0.927196,1,0,...,0,False,False,0,1,True,True,7,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,49.665393,49.787181,49.665393,NaN,3.190769,0.910918,False,0.913437,0,1,...,1,True,False,0,0,False,False,7,False,True
146,48.568928,48.796612,48.568928,0.116933,3.145652,0.932720,True,0.957072,0,0,...,1,False,False,0,0,True,False,6,True,False
147,47.566545,51.194584,47.566545,-0.650629,3.271737,0.928225,False,0.952135,1,0,...,1,True,False,0,0,True,True,5,True,False
148,47.087200,51.153969,47.087200,0.870876,3.150496,0.914168,True,0.954397,0,1,...,0,False,False,0,1,True,False,5,True,False
